In [8]:
# Configuration file for the LLM project
import pandas as pd
import numpy as np
from youtube_transcript_api import YouTubeTranscriptApi
from openai import OpenAI
import os
from dotenv import load_dotenv
import time
import re
import sys
sys.path.append('../src')

from food_buddy_api import get_food_buddy_recommendations

import warnings
warnings.filterwarnings("ignore")

Classes & Functions

In [9]:
class HellKitchenChef:
    def __init__(self):
        """
        Chef RAG powered by OpenAI API - Fast, reliable, and demo-ready!
        Requires OPENAI_API_KEY in .env file
        """
        print("🤖 Initializing Hell's Kitchen Chef with OpenAI...")
        
        # Load environment variables
        load_dotenv()
        
        # Get API key from environment
        self.api_key = os.getenv('OPENAI_API_KEY')
        if not self.api_key:
            raise ValueError("❌ OPENAI_API_KEY not found in environment variables. Add it to your .env file!")
        
        # Set up OpenAI client
        self.client = OpenAI(api_key=self.api_key)
        
        self.chef_persona = None
        self.model = "gpt-4o" 
        
        print("✅ Hell's Kitchen Chef ready 🔥")

    def create_persona_from_transcript(self, clean_transcript: str) -> str:
        """Create a chef personality paragraph from a transcript using OpenAI."""
        print("🔍 Creating persona from transcript...")
        
        # Truncate transcript if too long
        if len(clean_transcript) > 3000:
            clean_transcript = clean_transcript[:3000] + "..."
        
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {
                        "role": "system",
                        "content": "You are a chef personality analyzer. Create concise, accurate personality descriptions."
                    },
                    {
                        "role": "user", 
                        "content": f"""Analyze this chef transcript and create a personality description:

TRANSCRIPT (for reference only, do NOT repeat): {clean_transcript}

Instructions:
1. Write ONE cohesive paragraph describing how this chef talks, reacts, and behaves
2. Focus on their speaking style, attitude, temperament, and way of giving advice
3. Include their use of humor, sarcasm, encouragement, or harsh criticism
4. Describe how they would respond to cooking questions and critique recipes
5. Use descriptive language about their personality, not generic chef traits
6. Keep it under 200 words

Write the personality description:"""
                    }
                ],
                max_tokens=300,
                temperature=0.7
            )
            
            self.chef_persona = response.choices[0].message.content.strip()
            print("✅ Created Chef Persona:\n", self.chef_persona)
            return self.chef_persona
            
        except Exception as e:
            error_msg = f"❌ Persona generation failed: {str(e)}"
            print(error_msg)
            return error_msg

    def critique_recipe(self, recipe_data, user_query: str) -> str:
        """Generate a chef-style critique and recipe explanation using OpenAI."""
        if not self.chef_persona:
            return "❌ Create persona first using create_persona_from_transcript()"
        
        # Normalize recipe_data
        if isinstance(recipe_data, list):
            recipe_data = recipe_data[0] if recipe_data else {}
        elif not isinstance(recipe_data, dict):
            return "❌ Recipe data must be a dictionary or list of dictionaries"
        
        # Extract fields
        recipe_name = recipe_data.get("Name", recipe_data.get("name", "Unknown Recipe"))
        ingredients = recipe_data.get("ingredients_clean", recipe_data.get("ingredients", "No ingredients"))
        instructions = recipe_data.get("recipe_instructions_clean", recipe_data.get("instructions", "No instructions"))
        
        # Truncate to stay within token limits
        if len(ingredients) > 1000:
            ingredients = ingredients[:1000] + "..."
        if len(instructions) > 1500:
            instructions = instructions[:1500] + "..."
        
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {
                        "role": "system",
                        "content": f"You are a chef with this exact personality: {self.chef_persona}\n\nStay completely in character. Be authentic to this personality."
                    },
                    {
                        "role": "user",
                        "content": f"""User request: "{user_query}"

Recipe: {recipe_name}
Ingredients: {ingredients}
Instructions: {instructions}

Respond in character following this structure:
1. First, criticize their choice with your typical attitude
2. Then say: "But trust me, follow my instructions and you'll get a proper dish."
3. Then say: "If you use all these ingredients: [list key ingredients], you're going to end up with something amazing. I was thinking about {recipe_name}."
4. Finally, give the cooking instructions in your own words with your personality

Keep it conversational and authentic to your character. One response, stay in character throughout:"""
                    }
                ],
                max_tokens=600,
                temperature=0.8
            )
            
            return response.choices[0].message.content.strip()
            
        except Exception as e:
            error_msg = f"❌ Recipe critique failed: {str(e)}"
            print(error_msg)
            return error_msg

In [ ]:
def clean_transcript_text(transcript):
    """Transcript cleaner adding commas after each snippet"""
    
    # Add dot after each snippet, then join
    text_parts = []
    for snippet in transcript.snippets:
        clean_text = snippet.text.strip()
        if clean_text:
            text_parts.append(clean_text + ",")
    
    text = " ".join(text_parts)
    
    # Clean up
    text = re.sub(r'\[\xa0__\xa0\]', '', text)  # Remove censored parts
    text = re.sub(r'\[Music\]', '', text)       # Remove music tags
    text = re.sub(r'\s+', ' ', text).strip()    # Fix spacing
    
    # Capitalize first letter
    if text:
        text = text[0].upper() + text[1:]
    
    return text

In [11]:
# Query test
query = "pasta with chicken"
recipe = get_food_buddy_recommendations(query)
print(recipe)

[{'Name': 'Chicken Dijon & Pasta', 'Image_first': None, 'TotalTime_str': '1 hour 5 minutes', 'recipe_instructions_clean': 'Preheat oven to 400 degrees. Place chicken in shallow roasting pan. Mix mustard and dressing. Spoon half of mixture over chicken. Bake, uncovered, 40 minutes. Combine soup, 1/2 c water and remaining mustard mixture. Toss pasta wirth sauce, 2/3 c French Fried Onions, vegetables and parsley. Spoon mixture around chicken. Bake, uncovered, 15 minutes or until chicken is no longer pink in center. Sprinkle with remaining 2/3 c onions. Bake 1 minute or until onions are golden.', 'ingredients_clean': 'chicken, mustard, pasta, onion, tomatoes, zucchini, parsley', 'Calories': 567.7, 'similarity': 0.6422552343553112}, {'Name': 'Easy Lemon Pasta With Chicken', 'Image_first': None, 'TotalTime_str': '40 minutes', 'recipe_instructions_clean': 'Cook the pasta in a large pot of boiling salted water, until al dente. Drain well. Season chicken with salt and pepper. Heat a large grill

In [12]:
# Transcript test from YouTube video
video_id = "mdqb3fVqZgM"
api = YouTubeTranscriptApi()
transcript = api.fetch(video_id)

print(transcript)

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text="i'm gonna go [\xa0__\xa0] ballistic get a", start=0.08, duration=5.6), FetchedTranscriptSnippet(text='grip oh dear raj', start=2.32, duration=4.479), FetchedTranscriptSnippet(text='come here you', start=5.68, duration=2.56), FetchedTranscriptSnippet(text="it's raw", start=6.799, duration=5.681), FetchedTranscriptSnippet(text="it's [\xa0__\xa0] loud it's weird okay", start=8.24, duration=9.16), FetchedTranscriptSnippet(text='come on man what is that', start=12.48, duration=4.92), FetchedTranscriptSnippet(text='what what what what', start=17.76, duration=4.16), FetchedTranscriptSnippet(text='what is that', start=19.76, duration=5.04), FetchedTranscriptSnippet(text="ah it's like barbie's vomit it's like", start=21.92, duration=5.76), FetchedTranscriptSnippet(text="purple snot i swear to god it's [\xa0__\xa0]", start=24.8, duration=4.639), FetchedTranscriptSnippet(text="pinker inside that paris hilton's", start=27.68, duration=4.16

In [13]:
# Clean transcript test
transcript_clean = clean_transcript_text(transcript)
print(transcript_clean)

I'm gonna go ballistic get a, grip oh dear raj, come here you, it's raw, it's loud it's weird okay, come on man what is that, what what what what, what is that, ah it's like barbie's vomit it's like, purple snot i swear to god it's , pinker inside that paris hilton's, lipstick, just touch that fish there, just touch how dry it's got more , wrinkles on it than i have and i'm 44, years of age that should be , cooked fresh, thanks to there hurry up dear, barney yeah but i'm not dick face chef, yeah say that again i said i'm not dick, face you're pissed are you that last, bucket look at me look at me guys not as, pissed as i am you ah, donkey no you're cooking like a donkey i, didn't realize they shrunk down that, much chef, it's a bit like your brain not only is, it shrunk but it disappears, all of you look it's not it's just no, it's not about all it's not about that, oh, damn it pal a bit exploded all over my, face and like i have little tiny like, bits of halibut in my eyes that's what

In [14]:
# Instantiate the class
chef_rag = HellKitchenChef()
# Create a persona from the transcript
persona = chef_rag.create_persona_from_transcript(transcript_clean)
# Generate critique + recipe presentation
critique_and_recipe = chef_rag.critique_recipe(recipe[0], query)
print("\nChef Critique & Recipe:\n", critique_and_recipe)

🤖 Initializing Hell's Kitchen Chef with OpenAI...
✅ Hell's Kitchen Chef ready 🔥
🔍 Creating persona from transcript...
✅ Created Chef Persona:
 The chef exhibits a fiery and intense demeanor, characterized by a rapid-fire speaking style and a penchant for colorful, often harsh critiques. Their approach is highly animated, employing vivid analogies and biting sarcasm to emphasize their points, such as comparing poorly cooked food to "Barbie's vomit" or "Gandhi's flip-flop." This chef blends humor with criticism, using exaggerated expressions to drive home their dissatisfaction while maintaining an underlying sense of urgency and high standards. They are quick to express frustration, often escalating to yelling when faced with incompetence or culinary blunders, yet their outbursts are tinged with a unique brand of humor that lightens the severity of their reprimands. Despite their abrasive style, there's a clear desire to push their team towards excellence, albeit through tough love. When